In [1]:
import numpy as np
from matplotlib import pyplot as plt

import sys
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(14)

from sim_functions import *
from positioning import *
from system_parameters import *
from optimum_solution import *
from esnp_algorithms import *
from gt_power_control import *

In [2]:
# System constant parameters

num_ue, num_ap, num_ch = 4, 4, 1
                    
tot_band = 100e6 # [W]
env_constant = 10e-4
pl_constant = 4 

cov_side = 100 # [m]

pmin, pmax, step = 0, 30, 1 # [dBm]

In [3]:
# Study case dataset

ap_pos = ap_positions(num_ap, cov_side)

ue_pos = np.array([[22.538 + 20.333j], 
                   [56.679 + 32.188j], 
                   [76.551 + 14.688j], 
                   [26.595 + 70.239j]])

dis_matrix = distance(ue_pos, ap_pos)

shadowing_matrix = np.array([[5.3434e-2, 2.8731e-1, 1.9691e-2, 7.3013e-1], 
                             [3.2318, 1.5770, 2.6449e-1, 5.6379], 
                             [6.1470e-3, 1.1424, 2.6826e-1, 4.5709], 
                             [1.3485e-1, 4.6690e-1, 7.8250e-1, 1.6742]])

# I dunno why rayleigh_matrix has to have shape (num_ch, num_ue, num_ap) but for operation I'm doing, it works.

rayleigh_matrix = np.array([[1.248699, 3.248041, 0.772754, 0.708962], 
                            [0.498887, 0.104890, 0.647280, 0.940906], 
                            [0.382966, 0.682700, 1.891256, 0.327100], 
                            [0.065737, 0.649500, 1.981107, 1.259538]]).reshape(((num_ch, num_ue, num_ap)))

In [4]:
# Simulation

# Channel matrix is constant as distance, shadowing and fast fading matrixes are constant
channel_matrix = get_channel(dis_matrix, shadowing_matrix, np.sqrt(rayleigh_matrix))

# Set of all vectors of power possible in system
power_set = available_power_vec(pmin, pmax, step, num_ue)

# Noise power in system is constant
n_power = get_noise(tot_band, num_ch)

In [5]:
# ESNP algorithm

sinr_esnp_maxmin = esnp_maxmin(channel_matrix, pmax, pmin, step, 10, 0.5, 1000, n_power)

sinr_esnp_maxsum = esnp_maxsum(channel_matrix, pmax, pmin, step, 10, 0.5, 1000, n_power)

sinr_esnp_maxprod = esnp_maxproduct(channel_matrix, pmax, pmin, step, 10, 0.5, 1000, n_power)

print('maxmin: ', lin2db(np.min(sinr_esnp_maxmin[0])))
print(sinr_esnp_maxmin[1])
print('maxsum: ', lin2db(np.sum(sinr_esnp_maxsum[0])))
print(sinr_esnp_maxsum[1])
print('maxproduct: ', lin2db(np.sum(sinr_esnp_maxprod[0])*np.min(sinr_esnp_maxprod[0])))
print(sinr_esnp_maxprod[1])

maxmin:  -0.2898172049129422
[12. 30. 13. 22.]
maxsum:  53.85671749976571
[ 0.  0.  0. 30.]
maxproduct:  52.5002803408903
[ 0. 18.  7. 30.]


In [6]:
# Optimum values

# opt_maxmin = 0
# opt_summax = 0
# opt_maxprod = 0

# for vec in range(len(power_set)):

#     # Update the values in order to maximize each objective function
#     opt_maxmin = np.max([opt_maxmin, np.min(get_sinr(power_set[vec], channel_matrix, n_power))])
#     opt_summax = np.max([opt_summax, np.sum(get_sinr(power_set[vec], channel_matrix, n_power))])
#     opt_maxprod = np.max([opt_maxprod, (np.sum(get_sinr(power_set[vec], channel_matrix, n_power))
#                                         * np.min(get_sinr(power_set[vec], channel_matrix, n_power)))]) 

In [7]:
# print('maxmin: ', lin2db(opt_maxmin))
# print('maxsum: ', lin2db(opt_summax))
# print('maxprod: ', lin2db(opt_maxprod))

Optimum solution

maxmin:  -0.28882313317110436

maxsum:  53.85671749976571

maxprod:  52.518605576706754

In [8]:
# Game-based power allocation strategy

alpha = 0.5
epsilon = 1e-8

power_game = game_pas(20, alpha, num_ue, epsilon, channel_matrix)

sinr_game = get_sinr(power_game, channel_matrix, n_power)

print(lin2db(sinr_game))

1
[ 738.24722028  102.62666718   95.97924443 1973.91240033] 

2
[1730.44537869 4715.07177147 1710.50380118  174.61640819] 

3
[1404.09187244  852.68502028  695.83699696  274.42137228] 

4
[1462.50607402 1874.92718169  805.57381571  246.15705915] 

5
[1505.67043748 1330.39754156  804.83312834  242.63939095] 

6
[1458.03830707 1550.73621136  782.26490684  250.27416015] 

7
[1494.04969813 1448.87290504  802.94679457  243.71191796] 

8
[1471.14306868 1493.81462621  788.66933881  248.03750605] 

9
[1484.64837128 1473.38152996  797.40638621  245.40335817] 

10
[1477.03269919 1482.64783313  792.3516682   246.90444747] 

11
[1481.19401808 1478.39716107  795.1562856   246.07347807] 

12
[1478.96485852 1480.35785235  793.6361408   246.52114913] 

13
[1480.14021412 1479.44629544  794.44416525  246.28364375] 

14
[1479.52728364 1479.87290505  794.02009271  246.40794278] 

15
[1479.84406554 1479.67190339  794.24031617  246.34348495] 

16
[1479.68144107 1479.76719372  794.12682785  246.37665227] 

1

/home/matheuscarneiro/Documentos/power-control/sim_functions.py:10: RuntimeWarning: divide by zero encountered in log10
  return 10.0*np.log10(x)+30.0
/home/matheuscarneiro/Documentos/power-control/gt_power_control.py:24: RuntimeWarning: divide by zero encountered in divide
  inv_interference += 1/(dbm2lin(power_vec[i_ue]) * channel[:, i_ue, better_ch[ue]]**alpha)
/home/matheuscarneiro/Documentos/power-control/gt_power_control.py:27: RuntimeWarning: divide by zero encountered in divide
  inv_interference -= 1/interest
/home/matheuscarneiro/Documentos/power-control/gt_power_control.py:27: RuntimeWarning: invalid value encountered in subtract
  inv_interference -= 1/interest
/home/matheuscarneiro/Documentos/power-control/gt_power_control.py:30: RuntimeWarning: divide by zero encountered in divide
  gamma = interference / interest
